In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout, Flatten, Dense, BatchNormalization

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

In [2]:
img_width=256; img_height=256
batch_size=8

In [3]:
TRAINING_DIR = 'Kaggle_MRI_Tumor/Training'

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=30,
                                   zoom_range=0.4,
                                   horizontal_flip=True
                                   )

In [4]:
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    target_size=(img_height, img_width)
                                                    )

Found 2870 images belonging to 4 classes.


In [ ]:
train_generator.class_indices

In [ ]:
train_generator.labels

In [ ]:
train_generator.classes

In [ ]:
help(ImageDataGenerator.flow_from_directory)

In [5]:
VALIDATION_DIR = 'Kaggle_MRI_Tumor/Testing'

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [6]:
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=batch_size,
                                                              class_mode='categorical',
                                                              target_size=(img_height, img_width)
                                                             )

Found 394 images belonging to 4 classes.


In [ ]:
help(EarlyStopping)

In [ ]:
help(ModelCheckpoint)

In [7]:
callbacks = EarlyStopping(monitor='val_loss', patience=4, verbose=1, mode='auto')
best_model_file = '.../resnet50_drop_batch_best_weights_256.h5'
best_model = ModelCheckpoint(best_model_file, monitor='val_acc', verbose = 1, save_best_only = True)

In [ ]:
help(ResNet50)

In [8]:
#  wp = '.../weather_pred/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
resnet50_base = ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=(img_height, img_width,3))

In [ ]:
resnet50_base.get_layer

In [ ]:
help(Dense)

In [9]:
print('Adding new layers...')
output = resnet50_base.get_layer(index = -1).output  
output = Flatten()(output)
output = Dense(512,activation = "relu")(output)
output = BatchNormalization()(output)
output = Dropout(0.2)(output)
output = Dense(4, activation='softmax')(output)
print('New layers added!')

Adding new layers...
New layers added!


In [10]:
resnet50_model = Model(resnet50_base.input, output)
for layer in resnet50_model.layers[:-7]:
    layer.trainable = False

resnet50_model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 64, 64, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 64, 64, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 64, 64, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 64, 64, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 32, 32, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 32, 32, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 32, 32, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 32, 32, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 16, 16, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 16, 16, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 16, 16, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 16, 16, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 16, 16, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 16, 16, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 16, 16, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 16, 16, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

                                                                  'conv5_block1_3_bn[0][0]']      
                                                                                                  
 conv5_block1_out (Activation)  (None, 8, 8, 2048)   0           ['conv5_block1_add[0][0]']       
                                                                                                  
 conv5_block2_1_conv (Conv2D)   (None, 8, 8, 512)    1049088     ['conv5_block1_out[0][0]']       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 8, 8, 512)   2048        ['conv5_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_1_relu (Activatio  (None, 8, 8, 512)   0           ['conv5_block2_1_bn[0][0]']      
 n)       

In [ ]:
help(Adam)

In [ ]:
help(Model.compile)

In [11]:
resnet50_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics =['accuracy'])

In [ ]:
help(resnet50_model.fit)

In [12]:
history = resnet50_model.fit(train_generator,
                              epochs=30,
                              verbose=1,
                              validation_data=validation_generator,
                              callbacks = [callbacks, best_model]
                              )

Epoch 1/30
359/359 [==============================] - 650s 2s/step - loss: 0.9133 - accuracy: 0.6990 - val_loss: 1.2388 - val_accuracy: 0.6117
Epoch 2/30
359/359 [==============================] - 623s 2s/step - loss: 0.5570 - accuracy: 0.7983 - val_loss: 1.3553 - val_accuracy: 0.6269
Epoch 3/30
359/359 [==============================] - 632s 2s/step - loss: 0.4544 - accuracy: 0.8328 - val_loss: 1.1490 - val_accuracy: 0.6751
Epoch 4/30
359/359 [==============================] - 46290s 129s/step - loss: 0.4573 - accuracy: 0.8268 - val_loss: 0.9786 - val_accuracy: 0.7360
Epoch 5/30
359/359 [==============================] - 709s 2s/step - loss: 0.3981 - accuracy: 0.8561 - val_loss: 1.1152 - val_accuracy: 0.7107
Epoch 6/30
359/359 [==============================] - 701s 2s/step - loss: 0.3666 - accuracy: 0.8610 - val_loss: 0.9948 - val_accuracy: 0.7259
Epoch 7/30
359/359 [==============================] - 705s 2s/step - loss: 0.3869 - accuracy: 0.8585 - val_loss: 1.1073 - val_accuracy: 0.

In [ ]:
target_dir = '.../weather_pred/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
resnet50_model.save(target_dir + 'resnet50_model.h5')
resnet50_model.save_weights(target_dir + 'resnet50_weights.h5')

In [ ]:
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc))

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.plot(epochs, acc, 'r', label="Training Accuracy")
plt.plot(epochs, val_acc, 'b', label="Validation Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy of ResNet50')
plt.legend(loc='lower right')
plt.show()
fig.savefig('.../Accuracy_curve_resnet50_drop_batch_256.jpg')

In [ ]:
fig2 = plt.figure(figsize=(20,10))
plt.plot(epochs, loss, 'r', label="Training Loss")
plt.plot(epochs, val_loss, 'b', label="Validation Loss")
plt.legend(loc='upper right')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and validation loss of ResNet50')
fig2.savefig('.../Loss_curve_resnet50_drop_batch_256.jpg')